In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 2

In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, normalisation, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [7]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-04-27/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-05-19/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
#path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-12-19/"
files_start = 4547
files_end = 4713

In [8]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [9]:
def mad_based_outlier(points, thresh=3.5):
        """
        Returns a boolean array with True if points are outliers and False
        otherwise.

        Parameters:
        -----------
            points : An numobservations by numdimensions array of observations
            thresh : The modified z-score to use as a threshold. Observations with
                a modified z-score (based on the median absolute deviation) greater
                than this value will be classified as outliers.

        Returns:
        --------
            mask : A numobservations-length boolean array.

        References:
        ----------
            Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
            Handle Outliers", The ASQC Basic References in Quality Control:
            Statistical Techniques, Edward F. Mykytka, Ph.D., Editor.
        """
        if len(points.shape) == 1:
            points = points[:,None]
        median = np.median(points, axis=0)
        diff = np.sum((points - median)**2, axis=-1)
        diff = np.sqrt(diff)
        med_abs_deviation = np.median(diff)

        modified_z_score = 0.6745 * diff / med_abs_deviation

        return modified_z_score > thresh

#outlier rejection
def mad_outlier_rejection(df):
    data_sample_outlier = pd.DataFrame()
    columns = df.columns.tolist()
    for column in columns:
        data_sample_outlier[column] = df[column].where(~mad_based_outlier(df[column]), other=np.NaN)
    return data_sample_outlier

In [10]:
filenames = os.listdir(path_all)

In [11]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [12]:
#columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
#             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
        data_1_corr = mad_outlier_rejection(data_1_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_1_corr = mad_outlier_rejection(data_1_corr)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

/Users/marf/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # Remove the CWD from sys.path while we load stuff.
/Users/marf/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in divide
/Users/marf/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in greater


,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,,,
2016-05-19 15:09:00,1,blank sol,0.0,0.0,0.001513,0.001232,0.009708,0.001997,0.006525,0.002016,0.004293,0.000118,0.002666,0.001054,0.002309,0.001072,0.000928
2016-05-19 17:37:00,1,blank sol,0.0,0.0,0.000668,0.000462,0.004544,0.000903,0.002512,0.000784,0.002099,0.000053,0.002238,0.000907,0.001871,0.000933,0.000981
2016-05-19 17:37:00,2,blank sol,0.0,0.0,0.000773,0.000584,0.005194,0.001054,0.002953,0.000922,0.002876,0.000095,NaN,NaN,NaN,NaN,NaN
2016-05-19 17:37:00,3,blank sol,0.0,0.0,0.000848,0.000609,0.005518,0.001134,0.003224,0.000999,0.002946,0.000128,NaN,NaN,NaN,NaN,NaN
2016-05-19 17:37:00,4,blank sol,0.0,0.0,0.000642,0.000464,0.004369,0.000859,0.002365,0.000751,0.001910,0.000060,0.001846,0.000739,0.001577,0.000791,0.000787
2016-05-19 17:37:00,5,blank sol,0.0,0.0,0.000601,0.000409,0.004110,0.000770,0.002188,0.000678,0.001828,0.000045,0.001920,0.000767,0.001621,0.000797,0.000836
2016-05-19 17:37:00,6,blank sol,0.0,0.0,0.001017,0.000721,0.006328,0.001341,0.003775,0.001195,0.002784,0.000089,0.002092,0.000813,0.001781,0.000875,0.000869
2016-05-19 17:37:00,7,blank sol,0.0,0.0,0.000920,0.000635,0.005803,0.001242,0.003391,0.001068,0.002577,0.000083,0.002118,0.000844,0.001779,0.000891,0.000900
2016-05-19 17:37:00,8,blank sol,0.0,0.0,0.000724,0.000515,0.004762,0.000923,0.002640,0.000864,0.002429,0.000082,NaN,NaN,NaN,NaN,NaN


In [27]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
df_all_mean
#(df_all_mean["L1 (1)"] - (df_all_mean["L3 (1)"]/24.22*32.6))*1000

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1),sample
date,,,,,,,,,,,,,,,,
2016-05-19 15:09:00,0.0,0.0,0.001513,0.001232,0.009708,0.001997,0.006525,0.002016,0.004293,0.000118,0.002666,0.001054,0.002309,0.001072,0.000928,blank sol
2016-05-19 16:42:00,0.0,0.0,0.000801,0.000582,0.005255,0.001057,0.002962,0.000940,0.002406,0.000068,0.002080,0.000832,0.001765,0.000896,0.000871,blank sol
2016-05-19 17:06:00,0.0,0.0,0.000910,0.000685,0.005865,0.001240,0.003447,0.001083,0.002713,0.000069,0.002467,0.000930,0.002100,0.000957,0.000939,blank sol
2016-05-19 17:37:00,0.0,0.0,0.000687,0.000489,0.004612,0.000904,0.002536,0.000806,0.002121,0.000066,0.002021,0.000815,0.001706,0.000847,0.000855,blank sol
2016-05-19 18:03:00,0.0,0.0,0.000705,0.000517,0.004727,0.000930,0.002648,0.000834,0.002149,0.000063,0.002067,0.000855,0.001748,0.000888,0.000866,blank sol
2016-05-19 19:23:00,0.0,0.0,0.000770,0.000589,0.005025,0.001051,0.002897,0.000919,0.002263,0.000057,0.001882,0.000762,0.001602,0.000814,0.000765,blank sol
2016-05-19 20:43:00,0.0,0.0,0.000796,0.000581,0.005188,0.001089,0.003056,0.000962,0.002420,0.000058,0.001710,0.000686,0.001455,0.000729,0.000717,blank sol
2016-05-19 22:03:00,0.0,0.0,0.000884,0.000636,0.005628,0.001222,0.003418,0.001078,0.002516,0.000087,0.001736,0.000700,0.001479,0.000738,0.000722,blank sol
2016-05-19 23:23:00,0.0,0.0,0.001297,0.000990,0.007782,0.001803,0.004992,0.001571,0.003399,0.000076,0.001605,0.000638,0.001405,0.000646,0.000649,blank sol


In [27]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [11]:
df_all.to_csv(path_all + "blanks_outlier_corr.csv", mode='w', header=True)

In [12]:
df_all_mean.to_csv(path_all + "blanks_mean_outlier_corr.csv", mode='w', header=True)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")